In [48]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import cv2
from sklearn.utils import shuffle

In [2]:
train_dir = '../Traffic_Sign_Recognition_Project/'
test_dir = '../Traffic_Sign_Recognition_Project/'

IMG_SIZE = 75
BATCH_SIZE = 32
EPOCHS = 5

In [52]:
train_df = pd.read_csv("train.csv")
train_df = shuffle(train_df)
train_df.reset_index(inplace=True, drop=True)

In [53]:
train_df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,80,71,8,7,73,65,30,Train/30/00030_00004_00021.png
1,50,53,5,5,44,48,7,Train/7/00007_00047_00028.png
2,43,43,5,6,38,38,7,Train/7/00007_00025_00016.png
3,30,32,5,6,25,27,4,Train/4/00004_00056_00006.png
4,32,32,6,5,26,26,35,Train/35/00035_00025_00002.png


In [54]:
test_df = pd.read_csv('test.csv')
test_df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,53,54,6,5,48,49,16,Test/00000.png
1,42,45,5,5,36,40,1,Test/00001.png
2,48,52,6,6,43,47,38,Test/00002.png
3,27,29,5,5,22,24,33,Test/00003.png
4,60,57,5,5,55,52,11,Test/00004.png


In [55]:
test_df.dtypes

Width       int64
Height      int64
Roi.X1      int64
Roi.Y1      int64
Roi.X2      int64
Roi.Y2      int64
ClassId     int64
Path       object
dtype: object

In [56]:
test_df['ClassId'] = test_df['ClassId'].astype(str)
train_df['ClassId'] = train_df['ClassId'].astype(str)

In [57]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [58]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [59]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='Path',
    y_col='ClassId',
    batch_size=BATCH_SIZE,
    target_size=(IMG_SIZE, IMG_SIZE),
    seed=42,
    shuffle=True,
    subset='training', 
)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col="Path",
    y_col="ClassId",
    subset="validation",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(IMG_SIZE,IMG_SIZE)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='Path',
    y_col='ClassId',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=False,
    target_size=(IMG_SIZE, IMG_SIZE)    
)

Found 29407 validated image filenames belonging to 43 classes.
Found 9802 validated image filenames belonging to 43 classes.
Found 12630 validated image filenames belonging to 43 classes.


In [69]:
inception_model = tf.keras.applications.inception_v3.InceptionV3(include_top=False)
inception_model.trainable = True

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_layer')
x = inception_model(inputs, training=True)
x = layers.GlobalMaxPooling2D(name='global_avg_pool_layer')(x)
outputs = layers.Dense(43, activation='softmax', name='output_layer')(x)

new_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [70]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_1034 (Conv2D)            (None, None, None, 3 864         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1034 (Batch (None, None, None, 3 96          conv2d_1034[0][0]                
__________________________________________________________________________________________________
activation_1034 (Activation)    (None, None, None, 3 0           batch_normalization_1034[0][0]   
_______________________________________________________________________________________

In [71]:
for layer in inception_model.layers:
    print(layer.name, layer.trainable)

input_12 True
conv2d_1034 True
batch_normalization_1034 True
activation_1034 True
conv2d_1035 True
batch_normalization_1035 True
activation_1035 True
conv2d_1036 True
batch_normalization_1036 True
activation_1036 True
max_pooling2d_44 True
conv2d_1037 True
batch_normalization_1037 True
activation_1037 True
conv2d_1038 True
batch_normalization_1038 True
activation_1038 True
max_pooling2d_45 True
conv2d_1042 True
batch_normalization_1042 True
activation_1042 True
conv2d_1040 True
conv2d_1043 True
batch_normalization_1040 True
batch_normalization_1043 True
activation_1040 True
activation_1043 True
average_pooling2d_99 True
conv2d_1039 True
conv2d_1041 True
conv2d_1044 True
conv2d_1045 True
batch_normalization_1039 True
batch_normalization_1041 True
batch_normalization_1044 True
batch_normalization_1045 True
activation_1039 True
activation_1041 True
activation_1044 True
activation_1045 True
mixed0 True
conv2d_1049 True
batch_normalization_1049 True
activation_1049 True
conv2d_1047 True
con

In [72]:
new_model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 75, 75, 3)]       0         
_________________________________________________________________
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_avg_pool_layer (Globa (None, 2048)              0         
_________________________________________________________________
output_layer (Dense)         (None, 43)                88107     
Total params: 21,890,891
Trainable params: 21,856,459
Non-trainable params: 34,432
_________________________________________________________________


In [73]:
new_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [74]:
history = new_model.fit_generator(
    train_generator,
    validation_data=valid_generator,
    validation_steps=int(0.25*len(valid_generator)),
    epochs=15
)

Epoch 1/15
919/919 [==============================] - 137s 141ms/step - loss: 0.6097 - accuracy: 0.8484 - val_loss: 0.0871 - val_accuracy: 0.9778
Epoch 2/15
919/919 [==============================] - 128s 140ms/step - loss: 0.1225 - accuracy: 0.9697 - val_loss: 0.0469 - val_accuracy: 0.9889
Epoch 3/15
919/919 [==============================] - 121s 131ms/step - loss: 0.0820 - accuracy: 0.9818 - val_loss: 0.0776 - val_accuracy: 0.9790
Epoch 4/15
919/919 [==============================] - 131s 142ms/step - loss: 0.0566 - accuracy: 0.9865 - val_loss: 0.0599 - val_accuracy: 0.9848
Epoch 5/15
919/919 [==============================] - 165s 180ms/step - loss: 0.0578 - accuracy: 0.9877 - val_loss: 0.1022 - val_accuracy: 0.9774
Epoch 6/15
919/919 [==============================] - 174s 189ms/step - loss: 0.0791 - accuracy: 0.9822 - val_loss: 0.0347 - val_accuracy: 0.9914
Epoch 7/15
919/919 [==============================] - 181s 197ms/step - loss: 0.0282 - accuracy: 0.9929 - val_loss: 0.0413 -

In [75]:
new_model.evaluate(test_generator)

395/395 [==============================] - 18s 44ms/step - loss: 0.2879 - accuracy: 0.9573


[0.2879076302051544, 0.9573238492012024]